In [103]:
import pandas as pd

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

# imported the CSV data
df = pd.read_csv("Filtered_Data.csv", sep = ",")
df.columns  = [ column_name.split('.')[1] for column_name in df.columns ]

df.head()

,ts,impression_system_date,hour,conversion_status,cc_id_null_ind,ad_id,click_id,cc_id,hour_of_day,raw_publisher_id,...,click_conf_gmt_offset,click_state,click_city,ad_bid,ad_cpa_goal,ad_market_place_id,click_click_status,kmean_category_name,partner_id,version
0,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
1,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
2,20190106,2019-01-06,2,0,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,...,-600,al,birmingham,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
3,20190106,2019-01-06,4,0,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,...,-500,in,ft wayne,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
4,20190106,2019-01-06,6,0,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,...,-800,ca,mountain view,3.394224,NaN,1027,1,missing,8PRHGG6T9,1


In [104]:
# These columns contains redundant data, needs to be removed - only one type of value
# print (df.columns)
# print ('\n')
# print ('Below are the list of columns names having only single values in entire columns')

# for col in df.columns:
#     if(df[col].nunique() == 1):
#         print (col)

In [105]:
#Started removing above columns

In [106]:
# df = df.drop('cvr_base_data_table.cc_id_null_ind', axis = 1)

In [107]:
# df = df.drop('cvr_base_data_table.serving_key', axis = 1)
# df = df.drop('cvr_base_data_table.ad_partner_id', axis = 1)
# df = df.drop('cvr_base_data_table.ad_market_place_id', axis = 1)
# df = df.drop('cvr_base_data_table.click_click_status', axis = 1)
# df = df.drop('cvr_base_data_table.kmean_category_name', axis = 1)
# df = df.drop('cvr_base_data_table.version', axis = 1)

In [108]:
df.head()
df['ts'].value_counts()

20190110    23935
20190109    20771
20190107    20013
20190108    19250
20190104    14738
20190105    12964
20190106    12664
Name: ts, dtype: int64

In [109]:
# # Remove the timestamp data and impression system date data - results not dependent on the time frame
# df = df.drop('cvr_base_data_table.ts', axis = 1)
# df = df.drop('cvr_base_data_table.impression_system_date', axis = 1)

# # Server hour do not matters
# df = df.drop('cvr_base_data_table.hour', axis = 1)

# # Data have only 3 values of *au* region, and conversion status 0 for all of them
# df = df.drop('cvr_base_data_table.enriched_country', axis = 1)

In [110]:
# df.head()

In [111]:
# df['cvr_base_data_table.ad_cpa_goal'].value_counts()

In [112]:
# df['cvr_base_data_table.click_browser'].value_counts()

In [113]:
# df.info()
# # print (df['cvr_base_data_table.conversion_status'].value_counts().sum())

In [114]:
# dd = {'cvr_base_data_table.conversion_status' : {
#     'conversion' : 'sum',
#     'click' : 'count'
# }}
# dfg = df.groupby('cvr_base_data_table.click_browser').agg(dd).reset_index()
# dfg.columns = dfg.columns.droplevel(0)
# tot = dfg['click'].sum()
# dfg['cvr'] = dfg['conversion'] / dfg['click']
# dfg['click % of total data'] = dfg['click'] / tot * 100
# dfg = dfg.sort_values(by=['click % of total data'], ascending = False)
# dfg.columns

In [115]:
# tots = df.shape[0]
# for col in df.columns:
#     if col == 'cvr_base_data_table.conversion_status':
#         continue
#     dfg = df.groupby(col).agg(dd).reset_index()
#     dfg.columns = dfg.columns.droplevel(0)
    
#     dfg['cvr'] = dfg['conversion'] / dfg['click']
#     dfg['click percentage of total data'] = dfg['click'] / tots * 100
#     dfg = dfg.sort_values(by=['click percentage of total data'], ascending = False)
#     dfg.to_csv("Done/" + str(col) + ".csv")

In [116]:
# df['cvr_base_data_table.cc_id']

In [117]:
# df.columns

In [119]:
dd = {'conversion_status' : {
    'conversion' : 'sum',
    'click' : 'count'
}}
tots = df.shape[0]
dfg = df.groupby(['analyzer_name', 'ts']).agg(dd).reset_index()
dfg.columns = dfg.columns.droplevel(0)
dfg['cvr'] = dfg['conversion'] / dfg['click']
dfg['click % of total data'] = dfg['click'] / tot * 100
# dfg = dfg.sort_values(by=['click % of total data'], ascending = False)



,,,conversion,click,cvr,click % of total data
0,broadanalyserv2,20190104,3,33,0.090909,0.026541
1,broadanalyserv2,20190105,0,30,0.000000,0.024128
2,broadanalyserv2,20190106,2,21,0.095238,0.016890
3,broadanalyserv2,20190107,1,35,0.028571,0.028150
4,broadanalyserv2,20190108,3,44,0.068182,0.035388
5,broadanalyserv2,20190109,3,41,0.073171,0.032975
6,broadanalyserv2,20190110,0,29,0.000000,0.023324
7,categoryanalyser,20190104,7,54,0.129630,0.043431
8,categoryanalyser,20190105,1,41,0.024390,0.032975
9,categoryanalyser,20190106,4,52,0.076923,0.041822
